# Lab 4.1 - Wykrywanie palety kolorów

**Wykonanie rozwiązań: Marcin Przewięźlikowski**

https://github.com/mprzewie/ml_basics_course

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples

Wybierzmy ładne, kolorowe zdjęcie. Postarajmy się by miał kilka dominijących barw oraz jakieś anomalie (np. mały balonik na tle parku, albo samotny X-Wing na tle galaktyki).

Warunek kilku dominujących barw jest tylko jednym z powodów, dla których warto sięgnąć po klasykę komiksu :)

In [ ]:
image = cv2.imread("ds_1.jpg")
image = cv2.resize(image, (500, 800))
image = image[:,:,[2,1,0]]
image = image / 255
plt.figure(figsize=(6, 9))
plt.imshow(image)
plt.title("Okładka jednego z najlepszych komiksów wszechczasów")
plt.axis("off")
plt.show()


Potraktujmy każdy jego piksel jako obserwację w przestrzeni 3-D (po jednym wymiarze na każdy z kolorów). Zdecydujmy czy usuwamy ze zbioru duplikaty (piksele o takich samych wartościach RGB) - nasz wybór wpłynie na finalny wynik. 

In [ ]:
all_pixels = image.reshape(-1, 3)
unique_pixels = np.unique(all_pixels, axis=0)

Będe pracować na wszystkich pikselach.

Wykonajmy na takim zbiorze klasteryzację k-means, z następującymi założeniami:
* jako środków klastrów używamy istniejące elementy zbioru, a nie ich średnie (czyli jest to w praktyce k-medoids) - nie chcemy znaleźć kolorów, które nie wystąpiły na zdjęciu;
* dobieramy wartość stałej k używając dowolnej zaproponowanej przez siebie metody.

In [ ]:
kmeanses = []

fig = plt.figure(figsize=(15, 25))

for n_clusters in range(1, 11):
    kmeans = KMeans(n_clusters).fit(all_pixels)
    
    # zamieniam znalezione centra klastrów na punkty najbardziej im podobne z datasetu,
    # żeby otrzymać K-Medoids
    new_cluster_centers = []
    for c in kmeans.cluster_centers_:
        differences = unique_pixels - c
        differences_summed = (differences ** 2).sum(axis=1)
        min_difference = differences[np.argmin(differences_summed)]
        new_cluster_centers.append(c + min_difference)
    
    new_cluster_centers = np.array(new_cluster_centers)
    kmeans.cluster_centers_ = new_cluster_centers
    kmeanses.append(kmeans)
    
    cluster_indices = kmeans.predict(all_pixels)
    all_pixels_clustered = kmeans.cluster_centers_[cluster_indices].reshape(image.shape)    
    plt.subplot(5, 2, n_clusters)
    plt.title(f"n_clusters = {n_clusters}")
    plt.imshow(all_pixels_clustered)
    plt.axis("off")
    
plt.tight_layout()
plt.show()

In [ ]:
plt.title("Kwadratowa suma odległości punktów od swoich klastrów w zależności od liczby klastrów")
plt.plot(np.arange(len(kmeanses)), [k.inertia_ for k in kmeanses])
plt.xlabel("Liczba klastrów")
plt.xlabel("Suma kwadratowej odległości")
plt.show()

Z wykresu widać (z "elbow method"), że od $k=4$, zmiany w średnich odległościach punktów od ich klastrów nie są już tak duże, jak przy mniejszej liczbie klastrów.

Dodatkowo wizualizując klasteryzacje, $k \geq 5$ wydają się dawać ładne wizualne wyniki. Użyję więc dalej kmeans wytrenowanego dla $k=5$

In [ ]:
kmeans = kmeanses[4]

Prezentujemy uzyskaną paletę. 

In [ ]:
plt.title("Paleta kolorów znalezionych w k-means")
plt.imshow(np.array([kmeans.cluster_centers_]))
plt.show()

In [ ]:
sampled_pixels = unique_pixels[np.random.randint(0, len(unique_pixels), 10000)]
sampled_pixels_clusters = kmeans.predict(sampled_pixels)
clusters = kmeans.cluster_centers_
sampled_pixels_clustered = clusters[sampled_pixels_clusters]

In [ ]:
fig = plt.figure(figsize=(10, 10))
fig.suptitle("Przykładowe piksele z obrazka (po lewej) i kolory, do których zostały zmapowane przez k-means (po prawej)")
for i, c in enumerate(clusters):
    plt.subplot(1, len(clusters), i +1)
    pixels_of_cluster = sampled_pixels[sampled_pixels_clusters == i][:10]
    pixels_clustered = sampled_pixels_clustered[sampled_pixels_clusters == i][:10]
    original_and_clustered = np.hstack([
        pixels_of_cluster, pixels_clustered
    ]).reshape(-1, 2, 3)

    plt.axis("off")
    plt.imshow(original_and_clustered)

plt.show()

Wizualizujemy samą klasteryzację (np. rzutujemy punkty ze zbioru na 2D używając PCA, każdemu z nich środek malujemy na pierwotny kolor, a obwódkę na kolor klastra do którego był przyporządkowany).

In [ ]:
pca = PCA().fit(all_pixels)
sampled_pixels_pcad = pca.transform(sampled_pixels)
clusters_pcad = pca.transform(clusters)

In [ ]:
fig = plt.figure(figsize=(10,10))
fig.suptitle("Wizualizacja klasteryzacji bez centrów klastrów")
for i, (c, c_p) in enumerate(zip(clusters, clusters_pcad)):
    n_points = 20
    pixels_of_cluster = sampled_pixels[sampled_pixels_clusters == i][:n_points]
    pixels_pcad = sampled_pixels_pcad[sampled_pixels_clusters == i][:n_points]
    
    plt.scatter(pixels_pcad[:,0], pixels_pcad[:,1], c=[c for _ in pixels_pcad], s=400)
    plt.scatter(pixels_pcad[:,0], pixels_pcad[:,1], c=pixels_of_cluster, s=150)
plt.show()

Następnie na tej samej wizualizacji 2D pokazujemy centra znalezionych klastrów oraz wartość miary Silhouette dla każdego z punktów (jest zawsze z zakresu -1 do 1, można to zwizualizować skalą szarości). Jaki kolor miały oryginalnie punkty o najwyższym Silhouette, a jakie te o najniższym? Czy miara ta nadaje się do wykrywania punktów - anomalii?

Ponieważ $silhouette$ liczy się bardzo długo na pełnym zbiorze punktów, liczę je tylko na stworzonej wczesniej próbce.

In [ ]:
sampled_pixels_scores = silhouette_samples(sampled_pixels, sampled_pixels_clusters)

In [ ]:
fig = plt.figure(figsize=(15,15))
fig.suptitle("Wizualizacja klasteryzacji z centrami klastrów i wartością $silhouette$")
for i, (c, c_p) in enumerate(zip(clusters, clusters_pcad)):
    n_points = 20
    
    pixels_of_cluster = sampled_pixels[sampled_pixels_clusters == i][:n_points]
    pixels_pcad = sampled_pixels_pcad[sampled_pixels_clusters == i][:n_points]
    pixels_scores = sampled_pixels_scores[sampled_pixels_clusters == i][:n_points]
    
    plt.scatter(pixels_pcad[:,0], pixels_pcad[:,1], c=[c for _ in pixels_pcad], s=1100)
    plt.scatter(pixels_pcad[:,0], pixels_pcad[:,1], c="white", s=800)
    for (p_c, p_p, p_s) in zip(pixels_of_cluster, pixels_pcad, pixels_scores):       
        plt.scatter([p_p[0]], [p_p[1]], c=[p_c], s=600, marker=f"${'%.2f' % p_s}$")
    
    
    plt.scatter([c_p[0]], [c_p[1]], c="white", marker="D", s=800 )
    plt.scatter([c_p[0]], [c_p[1]], c=[c], marker="D", s=500 )
plt.show()

Można zaobserwować, że punkty bardziej oddalone od swoich klastrów i różniące się od nich kolorami mają widocznie niższe wartości $silhouette$. Pozwala to wysunąć hipotezę, że im niższa wartość $silhouette$ danego punktu, tym większa szansa że jest on punktem anomalicznym.